In [1]:
import torch
import numpy as np
#from transformers import BertTokenizer
#from transformers import BertModel
import pandas as pd
from torch.optim import Adam
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel

EPOCHS = 20
LR = 1e-6
batch_size = 2
dropout = 0

summaryCollumn = 'Syllabus_Legal' #qual coluna com o resumo no dataframe escolher para treinar
#summaryCollumn = 'Syllabus' #qual coluna com o resumo no dataframe escolher para treinar

#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

filePath = '/home/jaco/Projetos/landMarkClassification/data/onlyLandMarkWSyllabus.csv'
filePath_Train = '/home/jaco/Projetos/landMarkClassification/data/onlyLandMarkWSyllabus_Train.csv'
filePath_Test = '/home/jaco/Projetos/landMarkClassification/data/onlyLandMarkWSyllabus_Test.csv'
encodePath = '/home/jaco/Projetos/landMarkClassification/data/enCode.csv'

/home/jaco/.pyenv/versions/3.9.7/envs/landMarkClassification/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_labels(filePath,flagBurro):
    enCode = pd.read_csv(filePath)
    ag_news_label = enCode.to_dict(orient='split')
    tmpDic = {}

    if flagBurro:

        for n in ag_news_label['data']:
            tmpDic[n[0]] = n[1]

    else:

        for n in ag_news_label['data']:
            tmpDic[n[1]] = n[0]

    return tmpDic

def read_dfs(trainPath,testPath,summary):

    df_train = pd.read_csv(trainPath)
    df_train.rename(columns={summary:'text','encoded':'category'},inplace=True)

    df_test = pd.read_csv(testPath)
    df_test.rename(columns={summary:'text','encoded':'category'},inplace=True)

    return df_train,df_test

labels = get_labels(encodePath,False)

df_train,df_test = read_dfs(filePath_Train,filePath_Test,summaryCollumn)

print(df_test.head(5))

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        #self.labels = [labels[label] for label in df['category']]
        self.labels = np.array(list(df['category']))-1
        self.labels = list(self.labels)

        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

                                                text  \
0  Petitioner Ford, a black man charged with, int...   
1  A man was convicted of murder in the second de...   
2  After 21/2 years, appellant informed her emplo...   
3  Lloyd Gaines, a negro, was refused admission t...   
4  John F. Tinker, 15, and Christopher Eckhardt, ...   

                                      Syllabus_Legal  category  
0  The U.S. District Court for the Southern Distr...         4  
1  On December 6, 2019, the U.S. District Court f...         3  
2  On February 25, 2018, the U.S. District Court ...         2  
3  On December 12, 2018, the U.S. District Court ...         4  
4  The U.S. District Court for the Southern Distr...         2  


In [3]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout):

        super(BertClassifier, self).__init__()

        #self.bert = BertModel.from_pretrained('bert-base-cased')
        self.bert = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, len(labels) )
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [4]:
def train(model, train_data, val_data, learning_rate, epochs, batch_size):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size)

    use_cuda = torch.cuda.is_available()
    #use_cuda = False
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
model = BertClassifier(dropout=dropout)

train(model, df_train, df_test, LR, EPOCHS, batch_size)

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 177/177 [01:03<00:00,  2.79it/s]


Epochs: 1 | Train Loss:  0.693                 | Train Accuracy:  0.283                 | Val Loss:  0.701                 | Val Accuracy:  0.325


100%|██████████| 177/177 [01:09<00:00,  2.56it/s]


Epochs: 2 | Train Loss:  0.693                 | Train Accuracy:  0.283                 | Val Loss:  0.701                 | Val Accuracy:  0.325


100%|██████████| 177/177 [01:09<00:00,  2.55it/s]


Epochs: 3 | Train Loss:  0.692                 | Train Accuracy:  0.283                 | Val Loss:  0.701                 | Val Accuracy:  0.325


 11%|█▏        | 20/177 [00:08<01:03,  2.48it/s]


KeyboardInterrupt: 

In [ ]:
def evaluate(model, test_data,batch_size):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test,batch_size)

Test Accuracy:  0.475
